## Feature correlation analysis

## Context

with 400 + features, manu derived from similar signals, redudancy is inevitable. Highly correlated features add noise, slow down training, and can destabilize linear model. this analysis identifies clusters of correlated features for pruning

## Objective 
- compute correaltion metrices for numeric features
- identify highly correlated feature paris(|r| > 0.95)
- select candidates for removal based on correalatin and importance
- quantify potential dimensionality reduction


In [1]:
# import  library and load data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

train = pd.read_parquet(Path('../data/interim/train_merged.parquet'))
print(f'Data loaded: {train.shape}')

Data loaded: (590540, 434)


In [2]:
# numeric features
numeric_cols = train.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c not in ['Transactionid', 'isFraud']]

print(f'Numeric features : {len(numeric_cols)}')

# sample for speed
sample = train[numeric_cols].sample(n=50000, random_state=42)

Numeric features : 402


In [5]:
# V FEATURE CORRELATION
v_cols = [c for c in numeric_cols if c.startswith('V')]
print(f'v-features : {len(v_cols)}')

# compute correaltion matrix
v_corr = sample[v_cols].corr()

# find high correlation pairs
high_corr_pairs = []
for i in range(len(v_cols)):
    for j in range(1+1, len(v_cols)):
        if abs(v_corr.iloc[i, j ]) > 0.95:
            high_corr_pairs.append((v_cols[i], v_cols[j], v_corr.iloc[i,j]))


print(f'Highly correlated pairs |r| > 95 : {len(high_corr_pairs)}')
print('\nSample pairs :')
for f1, f2, r in high_corr_pairs[:10]:
    print(f'{f1} - {f2} : {r:.3f}')

v-features : 339
Highly correlated pairs |r| > 95 : 1604

Sample pairs :
V3 - V3 : 1.000
V4 - V4 : 1.000
V5 - V5 : 1.000
V6 - V6 : 1.000
V7 - V7 : 1.000
V8 - V8 : 1.000
V9 - V9 : 1.000
V10 - V10 : 1.000
V10 - V11 : 0.970
V11 - V10 : 0.970


## insight : V-features correlations clusters

from the correlation anyallysis , we identify dense clusters of redudancy :
- many v-features paris exceed r > 0.95 whic indicates near perfect correaltion
- these likely com from similiar aggregation windows or realted transformations in vesta's system
- keeping all of them adds noise and training overhead without imporving AUC

pruning strategy : for each correalted pair, drop the feature with lower target correlation or higher missingness to preserve the most predictive signal

In [6]:
# identify redundant features 

# target correaltion
target_corr = sample.join(train['isFraud']).corr()['isFraud'].drop('isFraud').abs()

# for each correalted pairm mark the one with lower target correaltionfor removal
to_remove = set()
for f1, f2, r in high_corr_pairs:
    if target_corr.get(f1, 0) < target_corr.get(f2, 0):
        to_remove.add(f1)
    else:
        to_remove.add(f2)